# Importing The Required Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
import difflib
#If spelling mistake happens by user while entering food name then it finds closest match of food given by user from our data
from nltk.stem.porter import PorterStemmer  #this gives rootword to a particular word
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#Gives similarity score for every movie as compared to any food item

import warnings

# Disable warning display
warnings.filterwarnings('ignore')

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

# Data Collection and Preprocessing

In [3]:
food_data=pd.read_csv("Cleaned_Food.csv")


In [4]:
#Shows First Five Rows of our Dataset
food_data.head()

,recipe_title,url,record_health,vote_count,rating,description,cuisine,course,diet,prep_time,...,instructions,author,tags,category,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,index
0,Roasted Peppers And Mushroom Tortilla Pizza Re...,https://www.archanaskitchen.com/roasted-pepper...,good,434,4.958525,is a quicker version pizza to satisfy your cr...,Mexican,Dinner,Vegetarian,15 M,...,To begin making the Roasted Peppers And Mushro...,Divya Shivaraman,Party Food Recipes|Tea Party Recipes|Mushroom ...,Pizza Recipes,NaN,NaN,NaN,NaN,NaN,0
1,Thakkali Gotsu Recipe,https://www.archanaskitchen.com/tomato-gotsu-r...,good,3423,4.932223,also known as the is a quick and easy to ma...,South Indian Recipes,Lunch,Vegetarian,10 M,...,To begin making Tomato Gotsu Recipe/ Thakkali ...,Archana Doshi,Vegetarian Recipes|Tomato Recipes|South Indian...,Indian Curry Recipes,NaN,NaN,NaN,NaN,NaN,1
2,Spicy Grilled Pineapple Salsa Recipe,https://www.archanaskitchen.com/spicy-grilled-...,good,2091,4.945959,Spicy Grilled Pineapple Salsa is a simple reci...,Mexican,Side Dish,Vegetarian,10 M,...,To begin making the Spicy Grilled Pineapple Sa...,Archana's Kitchen,Party Starter & Appetizer Recipes|Pineapple Re...,Mexican Recipes,NaN,NaN,NaN,NaN,NaN,2
3,Karwar Style Dali Thoy Recipe - Toor dal Curry,https://www.archanaskitchen.com/dali-thoy-reci...,good,990,4.888889,The is a quintessential of Konkani dish whic...,Coastal Karnataka,Side Dish,High Protein Vegetarian,5 M,...,To prepare Karwar Style Dali Thoy Recipe (Toor...,Jyothi Rajesh,Side Dish Recipes|South Indian Recipes|Indian ...,Indian Curry Recipes,NaN,NaN,NaN,NaN,NaN,3
4,Rajma Kofta In Milk And Poppy Seed Gravy Recipe,https://www.archanaskitchen.com/rajma-kofta-in...,good,345,4.828986,Koftas are traditional Indian recipes mostly w...,North Indian Recipes,Side Dish,High Protein Vegetarian,20 M,...,To begin making Rajma Kofta In Milk And Poppy ...,RUBY PATHAK,Side Dish Recipes|Indian Lunch Recipes|Office ...,Kofta Recipes,NaN,NaN,NaN,NaN,NaN,4


In [5]:
#Shows No of Columns and Rows in our Datasset
food_data.shape 

(1003, 22)

In [6]:
#Gives Detailed Information about our Dataset
food_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003 entries, 0 to 1002
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recipe_title   1003 non-null   object 
 1   url            1003 non-null   object 
 2   record_health  1003 non-null   object 
 3   vote_count     1003 non-null   int64  
 4   rating         1003 non-null   float64
 5   description    1002 non-null   object 
 6   cuisine        998 non-null    object 
 7   course         991 non-null    object 
 8   diet           994 non-null    object 
 9   prep_time      1002 non-null   object 
 10  cook_time      1002 non-null   object 
 11  ingredients    1002 non-null   object 
 12  instructions   1003 non-null   object 
 13  author         1003 non-null   object 
 14  tags           991 non-null    object 
 15  category       1003 non-null   object 
 16  Unnamed: 16    0 non-null      float64
 17  Unnamed: 17    0 non-null      float64
 18  Unnamed:

# Data Cleaning

In [7]:
#Remove unwanted sybols
food_data.replace({"ingredients":{"(":" ",")":" ","|":" ","/":" "} },inplace=True)

In [8]:
food_data["ingredients"].head()

0    Tortillas|Extra Virgin Olive Oil|Garlic|Mozzar...
1    Sesame (Gingelly) Oil|Mustard seeds (Rai/ Kadu...
2    Extra Virgin Olive Oil|Pineapple|White onion|R...
3    Arhar dal (Split Toor Dal)|Turmeric powder (Ha...
4    Rajma (Large Kidney Beans)|Cashew nuts|Sultana...
Name: ingredients, dtype: object

In [9]:
#selecting relevant features for recommendation
selected_features=['cuisine','diet','ingredients']
print(selected_features)

['cuisine', 'diet', 'ingredients']


In [10]:
#Replacing Null values with Null String
for feature in selected_features:
    food_data[feature]=food_data[feature].fillna('')

In [11]:
#Combining all the 5 Selected features
combined_features=food_data["cuisine"]+' '+food_data["diet"]+''+food_data["ingredients"]

# Stemming

In [12]:
port_stem=PorterStemmer() #Load PortorStemmer function to port_stem variable

In [13]:
def stemming(text):
  stemmed_text= re.sub('[^a-zA-Z]',' ',text) #Removing Things that are not alphabets like comma,Numbers and all. from text are replaced by space
  stemmed_text= stemmed_text.lower() #Convert all new text to Lower Case bcoz can create problem for ML model
  stemmed_text= stemmed_text.split() #Splitted and coverted to list 
  stemmed_text= [port_stem.stem(word) for word in stemmed_text if not word in stopwords.words('english') ] #Stemming the words that are not in stopwords
  stemmed_text=' '.join(stemmed_text)
  return stemmed_text

In [14]:
combined_features=combined_features.apply(stemming)

In [15]:
type(combined_features)

pandas.core.series.Series

In [16]:
combined_features.dtype

dtype('O')

In [17]:
print(combined_features)

0       mexican vegetariantortilla extra virgin oliv o...
1       south indian recip vegetariansesam gingelli oi...
2       mexican vegetarianextra virgin oliv oil pineap...
3       coastal karnataka high protein vegetarianarhar...
4       north indian recip high protein vegetarianrajm...
                              ...                        
998     south indian recip eggetarianridg gourd turai ...
999     continent vegetarianal purpos flour maida bake...
1000    north indian recip vegetariancabbag patta gobi...
1001    continent sugar free dietmango ripe milk chia ...
1002    andhra vegetarianric flour chana dal bengal gr...
Length: 1003, dtype: object


# Converting Text to Vectors

In [18]:
#Converting Text data to feature vectors We did this bcoz the cosine_similarity can work efficiently on numerical data rather than text data
vectorizer=TfidfVectorizer()
feature_vectors=vectorizer.fit_transform(combined_features)

In [19]:
print(feature_vectors)

  (0, 459)	0.24711891882514242
  (0, 92)	0.2678030395278881
  (0, 361)	0.3874249162209983
  (0, 488)	0.10839490391393175
  (0, 102)	0.18905957308968557
  (0, 521)	0.1384545819590397
  (0, 54)	0.18905957308968557
  (0, 310)	0.11163109762961555
  (0, 946)	0.24114592467523002
  (0, 571)	0.09835834250135848
  (0, 120)	0.20204947930866643
  (0, 453)	0.26115077344836857
  (0, 286)	0.11820034751266494
  (0, 485)	0.08689062753392225
  (0, 487)	0.3377305670750468
  (0, 930)	0.17064724842586756
  (0, 227)	0.172856114225279
  (0, 887)	0.401142327784492
  (0, 441)	0.26438696716405236
  (1, 344)	0.22795061872488737
  (1, 591)	0.07655166265627417
  (1, 316)	0.12804346513094395
  (1, 542)	0.09168163053897689
  (1, 685)	0.12787776220880268
  (1, 662)	0.2617003756831995
  :	:
  (1001, 577)	0.2787511936242154
  (1001, 650)	0.15704398847959739
  (1001, 155)	0.1703750542704551
  (1001, 443)	0.18267912382201865
  (1001, 601)	0.09986526707382096
  (1002, 14)	0.3357498293262418
  (1002, 855)	0.32411493729379

# Cosine Similarity

In [20]:
#Getting similarity scores using cosine similarity
similarity=cosine_similarity(feature_vectors)
#For each food it will compare with all food items

In [28]:
similarity.shape

(1003, 1003)

In [29]:
print(similarity)  #Schezwan Style Chinese Veg Samosa Recipe 

[[1.         0.04908719 0.32387886 ... 0.03275448 0.         0.03839097]
 [0.04908719 1.         0.06214044 ... 0.2795917  0.0112538  0.15333753]
 [0.32387886 0.06214044 1.         ... 0.1351963  0.         0.0750676 ]
 ...
 [0.03275448 0.2795917  0.1351963  ... 1.         0.02277665 0.22396971]
 [0.         0.0112538  0.         ... 0.02277665 1.         0.02338762]
 [0.03839097 0.15333753 0.0750676  ... 0.22396971 0.02338762 1.        ]]


In [30]:
#Getting the Food name from the User
food_name=input("Plz enter your Favourite Food : ")

Plz enter your Favourite Food : Panipuri


In [31]:
food_name=str(food_name)

In [32]:
#Creating List with all food names(made this to compare input with food names in data)
list_of_all_food=food_data["recipe_title"].tolist()
list_of_all_food=[str(food) for food in list_of_all_food]
print(list_of_all_food)

['Roasted Peppers And Mushroom Tortilla Pizza Recipe', 'Thakkali Gotsu Recipe ', 'Spicy Grilled Pineapple Salsa Recipe', 'Karwar Style Dali Thoy Recipe - Toor dal Curry', 'Rajma Kofta In Milk And Poppy Seed Gravy Recipe', 'Barnyard Millet Mushroom Biryani Recipe', 'Mexican Green Sauce Recipe ', 'Karnataka Style Avarekalu Chitranna Recipe', 'Schezwan Style Chinese Veg Samosa Recipe ', 'Pineapple Upside Down Cake Recipe', 'Matar Butter Masala Recipe', 'Cheesy Rice Poppers Recipe', 'Savory Shrimps On Skillet Recipe', 'Goan Sorak Curry Recipe', 'Toffee Banana Recipe ', 'Mango Tomato Bean Salsa Recipe', 'Coorg Style Kumbala Curry Recipe ', 'Kurkuri Bhindi And Onion Masala Recipe', 'Kerala Cherupayar Curry Recipe ', 'Kela Matar ki Sabzi Recipe ', 'Creamy Chicken Pesto Spaghetti Pasta Recipe', 'Hyderabadi Bagare Tamatar', 'Eggplant Rolls Recipe', 'Beans And Carrot Masala Pulav With Black Eyed Beans Recipe', 'Chettinad Kathirikai Chops Recipe ', 'Kodava Pacche Baale Barthad', 'Lebanese Chickpe

In [33]:
#Finding the close match for movie given by user
find_close_match=difflib.get_close_matches(food_name,list_of_all_food,cutoff=0.001)
print(find_close_match)

['Panjiri Recipe ', 'Hing Pani Puri Recipe', 'Kanchipuram Idlis Recipe']


In [34]:
close_match=find_close_match[0]
print(close_match)

Panjiri Recipe 


In [35]:
#Finding Index of movie with help of title
index_of_the_food=food_data[food_data.recipe_title==close_match]['index'].values[0]
print(index_of_the_food)

411


In [36]:
#Getting list of similar movies score
similarity_score=list(enumerate(similarity[index_of_the_food]))
#here list is to convert into list as can then acccess any element easily
#enumerate is to run a loop through all movies of that similarity index as similarity has list for every movie by it shows list of that specific index
print(similarity_score)

[(0, 0.0), (1, 0.04017414826157209), (2, 0.011432005777675995), (3, 0.057954325133633106), (4, 0.15148469289930605), (5, 0.08523765009874516), (6, 0.014376806236381922), (7, 0.03616517623249068), (8, 0.13667777386612476), (9, 0.036334310347951315), (10, 0.09811348831564151), (11, 0.0673318469576232), (12, 0.024432484171713165), (13, 0.09621756751673702), (14, 0.1220131835294326), (15, 0.02230720662813017), (16, 0.11948623896099625), (17, 0.11485413896287441), (18, 0.10298442807356958), (19, 0.08783827045465141), (20, 0.009681939456394728), (21, 0.09527007276081517), (22, 0.030217535529304153), (23, 0.08249181899194132), (24, 0.09675453211809076), (25, 0.029788960253265183), (26, 0.06333074386134926), (27, 0.07009711780294442), (28, 0.18923319014712736), (29, 0.08139742422576529), (30, 0.1430661055973718), (31, 0.06741301035043279), (32, 0.033669005606469515), (33, 0.04973874999522725), (34, 0.011339484334551017), (35, 0.11790216594156479), (36, 0.02771428360199881), (37, 0.035845459947

In [37]:
len(similarity_score)

1003

In [38]:
#Sorting the Movies based on similarity score
sorted_similar_food=sorted(similarity_score,key= lambda x:x[1], reverse=True )
#sorted is a function to sort and arrange elements in python
#taking similarity_score and lambda x means x is similarity_score and in it take 1st index that is of score as 0th index is of just index
#reverse=True meand in descending order
print(sorted_similar_food)

[(411, 1.0000000000000002), (983, 0.3572582942842372), (782, 0.35566000807890624), (981, 0.33101095437774347), (605, 0.3298822504790986), (696, 0.31631636004687935), (826, 0.3143848820854549), (955, 0.2967703695623185), (876, 0.291063338826029), (787, 0.2788886454051722), (369, 0.27693626678058536), (986, 0.27242654749936307), (248, 0.2656266958505299), (431, 0.26304469770148126), (102, 0.25607477515606697), (987, 0.25553267997226403), (939, 0.2313328955193908), (568, 0.22714454195338774), (317, 0.2270935804516233), (554, 0.22553535810898617), (919, 0.22243028871762924), (674, 0.21921940568044218), (486, 0.21372228153997963), (297, 0.21144724957768682), (632, 0.21128430662156172), (937, 0.2097403790205121), (956, 0.20897677510210394), (992, 0.20433405820374662), (253, 0.19864852492927568), (523, 0.19658038536980438), (634, 0.19550506479242952), (131, 0.19362155050644664), (536, 0.19351431261610086), (666, 0.19328439753351573), (688, 0.19162072998186236), (713, 0.19095451163871346), (56

In [39]:
#Print name of similar movies based on  index as we know them from sorted_similar_movies
print("Food Suggested for you : ")

i=1

for food in sorted_similar_food:
    index=food[0]
    
    title_of_the_food=food_data[food_data.index==index]['recipe_title'].values[0]
    if(i<10):
        print(i,".",title_of_the_food)
        i=i+1 

Food Suggested for you : 
1 . Panjiri Recipe 
2 . Badam Ki Kheer Recipe - Spiced Almond Milk
3 . Spiced Festival Drink With Saffron And Dry Fruits
4 . Dry Fruits Meetha Paneer Paratha Recipe
5 .  Gujarati Gol Papdi
6 . Gehun Ka Halwa Recipe 
7 . Khajur Satori Recipe 
8 . Whole Wheat Almond Bars Recipe
9 . Sweet and Spicy Stuffed Paneer In Kofta Curry Recipe


# Food Recomendation System

In [40]:
food_name=input("Plz enter your Favourite Food : ")

list_of_all_food=food_data["recipe_title"].tolist()

find_close_match=difflib.get_close_matches(food_name,list_of_all_food,cutoff=0.001)

close_match=find_close_match[0]

index_of_the_food=food_data[food_data.recipe_title==close_match]['index'].values[0]

similarity_score=list(enumerate(similarity[index_of_the_food]))

sorted_similar_food=sorted(similarity_score,key= lambda x:x[1], reverse=True )

print("Food Suggested for you : ")

i=1

for food in sorted_similar_food:
    index=food[0]
    
    title_of_the_food=food_data[food_data.index==index]['recipe_title'].values[0]
    recipe_of_the_food=food_data[food_data.index==index]['url'].values[0]
    if(i<11):
        print(i,".",title_of_the_food," : ",recipe_of_the_food)
        print("--------------------------------------------------------------------------")
        i=i+1      

Plz enter your Favourite Food : Pizza
Food Suggested for you : 
1 . Pizza Pinwheels Recipe  :  https://www.archanaskitchen.com/pizza-pinwheels-recipe
--------------------------------------------------------------------------
2 . Whole Wheat Pita Bread Recipe  :  https://www.archanaskitchen.com/whole-wheat-pita-bread-recipe
--------------------------------------------------------------------------
3 . Whole Wheat Crust Pizza With Zucchini And Basil Sauce  :  https://www.archanaskitchen.com/whole-wheat-crust-pizza-with-zucchini-and-basil-sauce
--------------------------------------------------------------------------
4 . Whole Wheat Vegetarian Stromboli Recipe  :  https://www.archanaskitchen.com/whole-wheat-vegetarian-stromboli-recipe
--------------------------------------------------------------------------
5 . Pull Apart Mini Pizza Boats Recipe  :  https://www.archanaskitchen.com/pull-apart-mini-pizza-boats-recipe
------------------------------------------------------------------------